# Finding Frequent Itermsets

## Code

In [ ]:
from __future__ import print_function
from pyspark.mllib.fpm import FPGrowth
from pyspark import SparkContext
import numpy as np
from numpy import array
import pandas as pd
from operator import add

sc = SparkContext(appName = "FindingFrequentItemsets")

###################################################### Load and prepare the data
data = sc.textFile(".......")
#print(data.take(10))

# get the discription of page
f = open("....",'r')
replaceMap = {}
for line in f:
    dic = line[:-1].split(' ')
    replaceMap[dic[0]] = dic[1]
f.close()

# count total records
NumRecord = data.map(lambda x: 1).reduce(add)

# count times of each page in each records
def countInEachRecord(line):
    pages = line.split(' ')
    if len(pages) >=1:
        pages.remove('')
    countDic = {}
    for page in pages:
        if page in countDic:
            countDic[page] += 1
        else:
            countDic[page] = 1
    return countDic.items() 
       
flatMapData = data.flatMap(countInEachRecord)
#print(flatMapData.take(10))

# find the most popular
countInTotal = flatMapData.reduceByKey(add).sortBy(lambda x:x[1],False)
changeCode = countInTotal.map(lambda x: (replaceMap[x[0]],x[1])).collect()
file = open("popular.txt","w")
for page in changeCode:
    print('----------------------------------')
    print(page[0] + '     ' + str(page[1]))
    file.write( page[0] + '     ' + str(page[1]) + '\n')
file.close()    

########################################################## do frequent itemsets analysis
def preFP(line):
    pages = line.split(' ')
    pages = list(np.unique(pages))
    result = []
    if len(pages) >=1:
        pages.remove('')
    for item in pages:
        result.append(replaceMap[item])
    return result
dataForFP = data.map(preFP)
#print(dataForFP.take(10))

############################################################# train the FPGrowth 
model = FPGrowth.train(dataForFP, minSupport=0.01, numPartitions=10)
result = model.freqItemsets().collect()
result.sort(key = lambda result : result[1],reverse=True)

# output
file = open("FindingFrequentItemsets.txt","w")
file.write('total record ' + str(NumRecord) + '\n')
for fi in result:
    print(fi)
    file.write(str(fi[0])+' ' + str(fi[1]) + '\n')
file.close()
sc.stop()

In [20]:
file = open('popular.txt','r')
for line in file:
    print(line) 
file.close()

frontpage     940469

news     452387

weather     439398

on-air     414928

sports     395880

local     386217

misc     305615

business     264899

msn-sports     216125

tech     207479

msn-news     196614

opinion     151409

health     131760

living     96817

summary     56576

bbs     25249

travel     16972



## Table 2: List of Frequent Itemset

In [21]:
file = open('FindingFrequentItemsets.txt','r')
for line in file:
    print(line)  
file.close()

total record 989818

[u'frontpage'] 313181

[u'on-air'] 217101

[u'news'] 175286

[u'tech'] 121948

[u'local'] 121719

[u'sports'] 119138

[u'business'] 112183

[u'weather'] 95615

[u'msn-news'] 90192

[u'misc'] 80514

[u'msn-sports'] 76948

[u'news', u'frontpage'] 74704

[u'living'] 57597

[u'health'] 50606

[u'business', u'frontpage'] 43178

[u'on-air', u'frontpage'] 40078

[u'local', u'frontpage'] 39183

[u'sports', u'frontpage'] 38826

[u'misc', u'frontpage'] 36419

[u'tech', u'frontpage'] 33364

[u'misc', u'on-air'] 33283

[u'living', u'frontpage'] 32722

[u'summary'] 29200

[u'local', u'news'] 27533

[u'misc', u'local'] 27029

[u'health', u'frontpage'] 26248

[u'news', u'on-air'] 25425

[u'tech', u'news'] 25292

[u'opinion'] 24987

[u'business', u'news'] 23453

[u'local', u'on-air'] 21445

[u'misc', u'news'] 19918

[u'sports', u'news'] 18964

[u'msn-news', u'local'] 18115

[u'msn-sports', u'sports'] 18013

[u'health', u'news'] 17680

[u'living', u'news'] 17366

[u'local', u'news'